In [27]:
import numpy as np
from numba import jit

# Funciones de prueba
@jit(nopython=True)
def Sphere(arr):
    suma = 0.0
    for x in arr:
        suma += x ** 2
    return suma

intervaloSphere = (-5.12, 5.12)

@jit(nopython=True)
def Ackley(arr):
    suma1 = 0.0
    suma2 = 0.0
    n = arr.shape[0]
    for x in arr:
        suma1 += x ** 2
        suma2 += np.cos(2 * np.pi * x)
    term1 = 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt(suma1 / n))
    term2 = np.exp(suma2 / n)
    return term1 - term2

intervaloAckley = (-30, 30)

@jit(nopython=True)
def Griewank(arr):
    suma = 0.0
    m = 1.0
    n = arr.shape[0]
    for i in range(1, n + 1):
        x_i = arr[i - 1]
        suma += (x_i ** 2) / 4000.0
        m *= np.cos(x_i / np.sqrt(i))
    return 1 + suma - m

intervaloGriewank = (-600, 600)

@jit(nopython=True)
def Rastrigin(arr):
    n = arr.shape[0]
    suma = 0.0
    for x in arr:
        suma += x ** 2 - 10 * np.cos(2 * np.pi * x)
    return 10 * n + suma

intervaloRastrigin = (-5.12, 5.12)

@jit(nopython=True)
def Rosenbrock(x):
    n = x.shape[0]
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

# Codificación y decodificación binaria
@jit(nopython=True)
def codifica_real(x, n_bit, intervalo):
    a, b = intervalo
    precision = (b - a) / ((2 ** n_bit) - 1)
    x = max(a, min(b, x))
    index = int((x - a) / precision)
    x_binario = np.zeros(n_bit, dtype=np.int64)
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1
    return x_binario

@jit(nopython=True)
def codifica_vector(vector_reales, n_bit, intervalo):
    vector_binario = np.empty((len(vector_reales), n_bit), dtype=np.int64)
    for i in range(len(vector_reales)):
        vector_binario[i] = codifica_real(vector_reales[i], n_bit, intervalo)
    return vector_binario

@jit(nopython=True)
def decodifica(x_cod, n_bits, intervalo):
    a, b = intervalo
    precision = (b - a) / ((2 ** n_bits) - 1)
    indice = 0
    for i in range(len(x_cod)):
        indice = (indice << 1) | x_cod[i]
    return a + indice * precision

@jit(nopython=True)
def decodifica_vector(vector_binario, n_bits, intervalo):
    vector_reales = np.empty(vector_binario.shape[0])
    for i in range(vector_binario.shape[0]):
        vector_reales[i] = decodifica(vector_binario[i], n_bits, intervalo)
    return vector_reales

# Función para generar una solución inicial
@jit(nopython=True)
def generar_solucion_inicial(n, n_bits, intervalo):
    solucion_reales = np.random.uniform(intervalo[0], intervalo[1], n)
    solucion_binaria = codifica_vector(solucion_reales, n_bits, intervalo)
    return solucion_binaria, solucion_reales

# Generar vecindad de vecinos
@jit(nopython=True)
def generar_vecindad(solucion_binaria, n_bits, intervalo):
    n_variables = solucion_binaria.shape[0]
    vecindad_binaria = np.empty((n_variables, n_variables, n_bits), dtype=np.int64)
    vecindad_reales = np.empty((n_variables, n_variables))

    for i in range(n_variables):
        vecino_binario = solucion_binaria.copy()
        bit_a_mutar = np.random.randint(0, n_bits)
        vecino_binario[i][bit_a_mutar] = 1 - vecino_binario[i][bit_a_mutar]
        vecindad_binaria[i] = vecino_binario
        vecindad_reales[i] = decodifica_vector(vecino_binario, n_bits, intervalo)

    return vecindad_binaria, vecindad_reales

# Función de recocido simulado con enfriamiento geométrico
@jit(nopython=True)
def recocido_simulado_enfriamiento_geom(n, n_bits, intervalo, T_ini, T_min, alpha, iteraciones, funcion_objetivo):
    solucion_binaria, solucion_reales = generar_solucion_inicial(n, n_bits, intervalo)
    s_mejor = solucion_reales
    f_mejor = funcion_objetivo(s_mejor)
    mejor_binario = solucion_binaria
    
    T = T_ini
    while T > T_min:
        for _ in range(iteraciones):
            vecindad_binaria, vecindad_reales = generar_vecindad(solucion_binaria, n_bits, intervalo)
            idx_vecino = np.random.randint(0, n)
            s_vecina = vecindad_reales[idx_vecino]
            f_vecina = funcion_objetivo(s_vecina)
            
            if f_vecina < f_mejor:
                solucion_binaria = vecindad_binaria[idx_vecino]
                solucion_reales = s_vecina
                f_mejor = f_vecina
                mejor_binario = solucion_binaria
            else:
                delta_f = f_vecina - f_mejor
                if np.random.rand() < np.exp(-delta_f / T):
                    solucion_binaria = vecindad_binaria[idx_vecino]
                    solucion_reales = s_vecina
        
        # Aplicamos el esquema de enfriamiento geométrico
        T *= alpha  # Actualizamos la temperatura multiplicándola por α
    
    return mejor_binario, s_mejor, f_mejor

# Función para realizar las repeticiones del recocido simulado
def recocido_simulado_geom_repeticiones(n, n_bits, intervalo, T_ini, T_min, alpha, iteraciones, funcion_objetivo, repeticiones=30, semilla=None):
    # Si no se proporciona semilla, generar una aleatoria
    if semilla is None:
        semilla = np.random.randint(1, 1000000)
    np.random.seed(semilla)  # Establecemos la semilla inicial
    
    print(f"Semilla inicial utilizada: {semilla}")
    
    resultados = []
    for rep in range(repeticiones):
        # Genera una nueva semilla para cada repetición
        semilla_repeticion = np.random.randint(1, 1000000)
        np.random.seed(semilla_repeticion)  # Establece la semilla para la repetición
        
        # Imprimir la semilla usada para cada repetición
        print(f"Repetición {rep+1}/{repeticiones}, semilla utilizada: {semilla_repeticion}")
        
        mejor_binario, s_mejor, f_mejor = recocido_simulado_enfriamiento_geom(
            n, n_bits, intervalo, T_ini, T_min, alpha, iteraciones, funcion_objetivo
        )
        resultados.append((mejor_binario, s_mejor, f_mejor))
    
    return resultados, semilla  # Devuelve los resultados y la semilla inicial

# Parámetros
n = 10  # Dimensión de la solución
n_bits = 20  # Número de bits por variable
intervalo = intervaloRosenbrock  # Intervalo para la función Sphere
T_ini = 1000  # Temperatura inicial
T_min = 100  # Temperatura mínima
alpha = 0.99 # Factor de enfriamiento geométrico
iteraciones = 1000  # Número de iteraciones por temperatura
repeticiones = 30  # Número de repeticiones
funcion_objetivo = Rosenbrock  # Cambia por la función que desees optimizar
# Ejecución
resultados, semilla_utilizada = recocido_simulado_geom_repeticiones(
    n, n_bits, intervaloSphere, T_ini, T_min, alpha, iteraciones, funcion_objetivo, repeticiones,
)
print(f"Semilla inicial utilizada: {semilla_utilizada}")
for i, (mejor_binario, s_mejor, f_mejor) in enumerate(resultados):
    print(f"Repetición {i + 1}/{repeticiones}:")
    print(f"  Mejor solución (binario): {mejor_binario}")
    print(f"  Mejor solución (reales): {s_mejor}")
    print(f"  Valor de la función objetivo: {f_mejor}\n")

Semilla inicial utilizada: 288845
Repetición 1/30, semilla utilizada: 306496
Repetición 2/30, semilla utilizada: 44807
Repetición 3/30, semilla utilizada: 382461
Repetición 4/30, semilla utilizada: 891588
Repetición 5/30, semilla utilizada: 820085
Repetición 6/30, semilla utilizada: 466198
Repetición 7/30, semilla utilizada: 833637
Repetición 8/30, semilla utilizada: 485279
Repetición 9/30, semilla utilizada: 282459
Repetición 10/30, semilla utilizada: 200480
Repetición 11/30, semilla utilizada: 310088
Repetición 12/30, semilla utilizada: 622503
Repetición 13/30, semilla utilizada: 741861
Repetición 14/30, semilla utilizada: 965224
Repetición 15/30, semilla utilizada: 934944
Repetición 16/30, semilla utilizada: 768344
Repetición 17/30, semilla utilizada: 881619
Repetición 18/30, semilla utilizada: 234151
Repetición 19/30, semilla utilizada: 552348
Repetición 20/30, semilla utilizada: 995608
Repetición 21/30, semilla utilizada: 967583
Repetición 22/30, semilla utilizada: 196666
Repetici

In [31]:
import numpy as np
from numba import jit

# Funciones de prueba
@jit(nopython=True)
def Sphere(arr):
    suma = 0.0
    for x in arr:
        suma += x ** 2
    return suma                                           

intervaloSphere = (-5.12, 5.12)

@jit(nopython=True)
def Ackley(arr):
    suma1 = 0.0
    suma2 = 0.0
    n = arr.shape[0]
    for x in arr:
        suma1 += x ** 2
        suma2 += np.cos(2 * np.pi * x)
    term1 = 20 + np.exp(1) - 20 * np.exp(-0.2 * np.sqrt(suma1 / n))
    term2 = np.exp(suma2 / n)
    return term1 - term2

intervaloAckley = (-30, 30)

@jit(nopython=True)
def Griewank(arr):
    suma = 0.0
    m = 1.0
    n = arr.shape[0]
    for i in range(1, n + 1):
        x_i = arr[i - 1]
        suma += (x_i ** 2) / 4000.0
        m *= np.cos(x_i / np.sqrt(i))
    return 1 + suma - m

intervaloGriewank = (-600, 600)

@jit(nopython=True)
def Rastrigin(arr):
    n = arr.shape[0]
    suma = 0.0
    for x in arr:
        suma += x ** 2 - 10 * np.cos(2 * np.pi * x)
    return 10 * n + suma

intervaloRastrigin = (-5.12, 5.12)

@jit(nopython=True)
def Rosenbrock(x):
    n = x.shape[0]
    suma = 0.0
    for i in range(n - 1):
        suma += 100 * (x[i + 1] - (x[i]) ** 2) ** 2 + (x[i] - 1) ** 2
    return suma

intervaloRosenbrock = (-2.048, 2.048)

# Codificación y decodificación binaria
@jit(nopython=True)
def codifica_real(x, n_bit, intervalo):
    a, b = intervalo
    precision = (b - a) / ((2 ** n_bit) - 1)
    x = max(a, min(b, x))
    index = int((x - a) / precision)
    x_binario = np.zeros(n_bit, dtype=np.int64)
    for i in range(n_bit - 1, -1, -1):
        x_binario[i] = index & 1
        index >>= 1
    return x_binario

@jit(nopython=True)
def codifica_vector(vector_reales, n_bit, intervalo):
    vector_binario = np.empty((len(vector_reales), n_bit), dtype=np.int64)
    for i in range(len(vector_reales)):
        vector_binario[i] = codifica_real(vector_reales[i], n_bit, intervalo)
    return vector_binario

@jit(nopython=True)
def decodifica(x_cod, n_bits, intervalo):
    a, b = intervalo
    precision = (b - a) / ((2 ** n_bits) - 1)
    indice = 0
    for i in range(len(x_cod)):
        indice = (indice << 1) | x_cod[i]
    return a + indice * precision

@jit(nopython=True)
def decodifica_vector(vector_binario, n_bits, intervalo):
    vector_reales = np.empty(vector_binario.shape[0])
    for i in range(vector_binario.shape[0]):
        vector_reales[i] = decodifica(vector_binario[i], n_bits, intervalo)
    return vector_reales

# Función para generar una solución inicial
@jit(nopython=True)
def generar_solucion_inicial(n, n_bits, intervalo):
    solucion_reales = np.random.uniform(intervalo[0], intervalo[1], n)
    solucion_binaria = codifica_vector(solucion_reales, n_bits, intervalo)
    return solucion_binaria, solucion_reales

# Generar vecindad de vecinos
@jit(nopython=True)
def generar_vecindad(solucion_binaria, n_bits, intervalo):
    n_variables = solucion_binaria.shape[0]
    vecindad_binaria = np.empty((n_variables, n_variables, n_bits), dtype=np.int64)
    vecindad_reales = np.empty((n_variables, n_variables))

    for i in range(n_variables):
        vecino_binario = solucion_binaria.copy()
        bit_a_mutar = np.random.randint(0, n_bits)
        vecino_binario[i][bit_a_mutar] = 1 - vecino_binario[i][bit_a_mutar]
        vecindad_binaria[i] = vecino_binario
        vecindad_reales[i] = decodifica_vector(vecino_binario, n_bits, intervalo)

    return vecindad_binaria, vecindad_reales

# Función de recocido simulado con enfriamiento lineal
@jit(nopython=True)
def recocido_simulado_enfriamiento_lineal(n, n_bits, intervalo, T_ini, T_min, eta, iteraciones, funcion_objetivo):
    solucion_binaria, solucion_reales = generar_solucion_inicial(n, n_bits, intervalo)
    s_mejor = solucion_reales
    f_mejor = funcion_objetivo(s_mejor)
    mejor_binario = solucion_binaria
    
    T = T_ini
    k = 0  # Inicializamos el contador de iteraciones

    while T > T_min:
        for _ in range(iteraciones):
            vecindad_binaria, vecindad_reales = generar_vecindad(solucion_binaria, n_bits, intervalo)
            idx_vecino = np.random.randint(0, n)
            s_vecina = vecindad_reales[idx_vecino]
            f_vecina = funcion_objetivo(s_vecina)
            
            if f_vecina < f_mejor:
                solucion_binaria = vecindad_binaria[idx_vecino]
                solucion_reales = s_vecina
                f_mejor = f_vecina
                mejor_binario = solucion_binaria
            else:
                delta_f = f_vecina - f_mejor
                if np.random.rand() < np.exp(-delta_f / T):
                    solucion_binaria = vecindad_binaria[idx_vecino]
                    solucion_reales = s_vecina
        
        k += 1
        T = T_ini - eta * k
    
    return mejor_binario, s_mejor, f_mejor

# Función para realizar las repeticiones del recocido simulado con enfriamiento lineal
def recocido_simulado_repeticiones(n, n_bits, intervalo, T_ini, T_min, eta, iteraciones, funcion_objetivo, repeticiones=30, semilla=None):
    # Si no se proporciona semilla, generar una aleatoria
    if semilla is None:
        semilla = np.random.randint(1, 1000000)
    np.random.seed(semilla)  # Establecemos la semilla inicial
    
    print(f"Semilla inicial utilizada: {semilla}")
    
    resultados = []
    for rep in range(repeticiones):
        # Genera una nueva semilla para cada repetición
        semilla_repeticion = np.random.randint(1, 1000000)
        np.random.seed(semilla_repeticion)  # Establece la semilla para la repetición
        
        # Imprimir la semilla usada para cada repetición
        print(f"Repetición {rep + 1}/{repeticiones}, semilla utilizada: {semilla_repeticion}")
        
        mejor_binario, s_mejor, f_mejor = recocido_simulado_enfriamiento_lineal(
            n, n_bits, intervalo, T_ini, T_min, eta, iteraciones, funcion_objetivo
        )
        resultados.append((mejor_binario, s_mejor, f_mejor))
    
    return resultados, semilla  # Devuelve los resultados y la semilla inicial

# Parámetros
n = 10  # Dimensión de la solución
n_bits = 20  # Número de bits por variable
T_ini = 1000  # Temperatura inicial
T_min = 50  # Temperatura mínima
intervalo= intervaloRosenbrock
eta = 0.99  # Tasa de enfriamiento
iteraciones = 1000  # Número de iteraciones por temperatura
repeticiones = 30  # Número de repeticiones
funcion_objetivo = Rosenbrock  # Cambia por la función que desees optimizar

# Ejecución
resultados, semilla_utilizada = recocido_simulado_repeticiones(
    n, n_bits, intervaloRosenbrock, T_ini, T_min, eta, iteraciones, funcion_objetivo, repeticiones
)
print(f"Semilla inicial utilizada: {semilla_utilizada}")

# Imprimir resultados
# Imprimir resultados
for i, (mejor_binario, s_mejor, f_mejor) in enumerate(resultados):
    print(f"Repetición {i + 1}/{repeticiones}:")
    print(f"  Mejor solución (binario): {mejor_binario}")
    print(f"  Mejor solución (reales): {s_mejor}")
    print(f"  Valor de la función objetivo: {f_mejor}\n")


Semilla inicial utilizada: 608677
Repetición 1/30, semilla utilizada: 994136
Repetición 2/30, semilla utilizada: 102724
Repetición 3/30, semilla utilizada: 743416
Repetición 4/30, semilla utilizada: 297355
Repetición 5/30, semilla utilizada: 477437
Repetición 6/30, semilla utilizada: 357303
Repetición 7/30, semilla utilizada: 483527
Repetición 8/30, semilla utilizada: 89848
Repetición 9/30, semilla utilizada: 736301
Repetición 10/30, semilla utilizada: 703704
Repetición 11/30, semilla utilizada: 128373
Repetición 12/30, semilla utilizada: 175886
Repetición 13/30, semilla utilizada: 451166
Repetición 14/30, semilla utilizada: 280423
Repetición 15/30, semilla utilizada: 436103
Repetición 16/30, semilla utilizada: 701321
Repetición 17/30, semilla utilizada: 320157
Repetición 18/30, semilla utilizada: 488860
Repetición 19/30, semilla utilizada: 2640
Repetición 20/30, semilla utilizada: 82916
Repetición 21/30, semilla utilizada: 450613
Repetición 22/30, semilla utilizada: 846877
Repetición 